<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#LDA主题模型分析" data-toc-modified-id="LDA主题模型分析-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>LDA主题模型分析</a></span><ul class="toc-item"><li><span><a href="#预处理" data-toc-modified-id="预处理-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>预处理</a></span></li><li><span><a href="#LDA-分析" data-toc-modified-id="LDA-分析-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>LDA 分析</a></span></li><li><span><a href="#输出每个主题对应词语" data-toc-modified-id="输出每个主题对应词语-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>输出每个主题对应词语</a></span></li><li><span><a href="#输出每篇文章对应主题" data-toc-modified-id="输出每篇文章对应主题-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>输出每篇文章对应主题</a></span></li><li><span><a href="#可视化" data-toc-modified-id="可视化-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>可视化</a></span></li></ul></li></ul></div>

# LDA主题模型分析

主题模型能够自动把文本语料库编码为一组具有实质性意义的类别，这些类别称为主题。  
LDA-隐含狄利克雷分布

## 预处理

In [1]:
import os
import pandas as pd
import re
import jieba
import jieba.posseg as psg

In [2]:
output_path = r'../result'
file_path = r'../data'
os.chdir(file_path)
data=pd.read_excel(r"TrainData_1.xlsx")#content type
os.chdir(r'D:\JupyterProject\GraduateProject\project')
dic_file = r"../stop_dic/user_dict.txt"
stop_file = r"../stop_dic/hit_stopwords.txt"
os.chdir(output_path)

In [3]:
def chinese_word_cut(mytext):
    jieba.load_userdict(dic_file)
    jieba.initialize()
    try:
        stopword_list = open(stop_file,encoding ='utf-8')
    except:
        stopword_list = []
        print("error in stop_file")
    stop_list = []
    flag_list = ['n','nz','vn']
    for line in stopword_list:
        line = re.sub(u'\n|\\r', '', line)
        stop_list.append(line)
    
    word_list = []
    #jieba分词
    seg_list = psg.cut(mytext)
    for seg_word in seg_list:
        word = re.sub(u'[^\u4e00-\u9fa5]','',seg_word.word)
        #word = seg_word.word  #如果想要分析英语文本，注释这行代码，启动下行代码
        find = 0
        for stop_word in stop_list:
            if stop_word == word or len(word)<2:     #this word is stopword
                    find = 1
                    break
        if find == 0 and seg_word.flag in flag_list:
            word_list.append(word)      
    return (" ").join(word_list)

In [4]:
data["content_cutted"] = data.content.apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\20866\AppData\Local\Temp\jieba.cache
Loading model cost 0.548 seconds.
Prefix dict has been built successfully.


## LDA 分析

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [6]:
def print_top_words(model, feature_names, n_top_words):
    tword = []
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        topic_w = " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        tword.append(topic_w)
        print(topic_w)
    return tword

In [7]:
n_features = 1000 #提取1000个特征词语
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(data.content_cutted)

In [8]:
n_topics = 3
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=50,
                                learning_method='batch',
                                learning_offset=50,
#                                 doc_topic_prior=0.1,
#                                 topic_word_prior=0.01,
                               random_state=0)
lda.fit(tf)

LatentDirichletAllocation(learning_offset=50, max_iter=50, n_components=8,
                          random_state=0)

## 输出每个主题对应词语

In [9]:
n_top_words = 10
tf_feature_names = tf_vectorizer.get_feature_names_out()
topic_word = print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
有点 问题 方式 课件 逻辑 初学者 例子 建议 水平 精彩 代码 思路 同学 数据 证书 北大 团队 考试 概念 生活 个人 影响 收益 节奏 语速
Topic #1:
教学 生动 形象 地方 实际 效果 教师 资料 学生 整体 程序 专业 教学内容 网络 课堂 思维 课本 练习题 重点 例子 教学效果 做题 魅力 真棒 语言
Topic #2:
课程 数据结构 算法 学校 大学 代码 教授 总体 网课 慕课 过程 习题 新手 小白 设置 平台 价值 基本 让我学 兴趣 计算机专业 印象 工作 魅力 精品
Topic #3:
视频 课程内容 学生 课堂 声音 重点 细节 自学 五星 答案 背景音乐 方面 字幕 小白 国家精品 资源 章节 测验 学期 干货 优质 预期 学校 太棒 逻辑性
Topic #4:
老师 用心 平台 举例 细心 慕课 风格 制作 讲授 教导 普通话 网课 照本宣科 框架 节奏 例题 印象 实例 过程 一节课 认真负责 重点 代码 工作 同学
Topic #5:
操作系统 知识点 计算机 姥姥 题目 棒棒 学会 能力 部分 习题 考研 教授 软件 进程 分析 小白 编程 评价 同学 代码 精品 成绩 哈工大 工作 测验
Topic #6:
全面 时间 作业 系统 合理 理论 编程 原理 教材 结构 大家 质量 老师上课 概念 实验 体验 设计 同学们 测试 体系 讲授 考试 有所 课程设计 交流
Topic #7:
内容 感觉 基础 语言 全面 认真负责 慕课 基本 让我学 同学 总结 教程 课本 精简 书本 原因 风趣 节奏 形式 期末考试 空间 重点 方面 情况 编程


## 输出每篇文章对应主题

In [10]:
import numpy as np
topics=lda.transform(tf)

In [11]:
topic = []
for t in topics:
    topic.append("Topic #"+str(list(t).index(np.max(t))))
data['概率最大的主题序号']=topic
data['每个主题对应概率']=list(topics)
data.to_excel("data_topic.xlsx", index=False)

## 可视化

In [21]:
# import pyLDAvis
# import pyLDAvis.sklearn
# import pyLDAvis.gensim

In [22]:
# pyLDAvis.enable_notebook()
# pic = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)
# pyLDAvis.display(pic)
# pyLDAvis.save_html(pic, 'lda_pass'+str(n_topics)+'.html')
# pyLDAvis.display(pic)

In [23]:
# pip list --outdated

In [24]:
# pip install --upgrade pyLDAvis

In [25]:
# import pyLDAvis.gensim_models